# Homework 4 - Hard coding

## Pt 1 - Hashing

#### In the first part we deal with hash function and HLL ( HyperLogLog) algorithm to estimate the cardinality of a dataset.


In [1]:
import pandas as pd
import json
from collections import defaultdict
import numpy as np

In [2]:
df = pd.read_table('hash.txt')

#### First we define the hash function. For each element of the table we choose to transform its digits in the correspondent ASCII code. After that we sum the ASCII codes of the element and we apply the module operation. 


In [3]:
size = 49979687 # we choose a prime number of order of magnitude comparable to our table

def hash_f(name): 
    list_name = list(name) 
    ascii_sum = 0 
    for l in list_name: 
        add = ord(l) # take the corresponding ASCII code
        ascii_sum = ascii_sum + add 
    ascii_mod = ascii_sum % size # we use the module operation to hash 
    return ascii_mod 

#### After that we implement our hash tables

In [4]:
h_list = []
for k in range(size):
    h_list.append(k)
hash_table = {key: [] for key in h_list}  # we make an empty dictionary that will be our hash table 

h_list_b = []
for k in range(size):
    h_list_b.append(k)
hash_table_bin = {key: [] for key in h_list_b} # we make another empy dictionary for the elements of our table written in binary

In [5]:
for k in df['844082e02a27ddee8d99ea1af94a2969']:
    hash_table[hash_f(k)].append(k) # # hash table with the colliding elements in list

In [6]:
for k in df['844082e02a27ddee8d99ea1af94a2969']:
    hash_table_bin[hash_f(k)].append(bin(int(k,16))[3:]) # # hash table with the colliding elements in list

In [7]:
for key in hash_table:
    hash_table[key] = list(set(hash_table[key])) # to take out repeated elements


In [8]:
for key in hash_table_bin:
    hash_table_bin[key] = list(set(hash_table_bin[key])) # to take out repeated elements

#### After implementing our hash function we get the hash tables. Now we pass to the Hyperloglog algorithm. In the Hyperloglog algorithm we estimate the cardinality of the table using the number of zeros at the end of the elements in the binary hash table. Before doing that we have to divide the table in subgroups given by the first 5 digits (we choose to have 32 different subgroups). 
 

In [9]:
r0_table_bin={}
for k in hash_table_bin:
        r0_table_bin[k] = [(len(j) - len(j.rstrip('0'))) for j in hash_table_bin[k]] 
# counts the numbers of consecutives zeroes on the right for each element in the hash table



In [ ]:
l0_table_bin={}
for k in hash_table_bin:    
    l0_table_bin[k] = [int(j[:5],2) for j in hash_table_bin[k]]
# takes the first 5 digits for assigning the subgroup

#### now we implement our HLL dictionary to store the highest value in each subgroup 

In [10]:
hll_d = {} # Hyperloglog dictionary

keys = range(32)
l = [0]*32
for i in keys:
    for x in l:
        hll_d[i] = x
# we have now a dictionary with 32 keys
print(hll_d)

{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0}


In [11]:
for key, value in l0_table_bin.items(): # the cicle runs over the dictionary l0_table_bin
    i = 0
    while i < len(value): # the cicle runs over the lists in the dictionary l0_table_bin
        r0_v = r0_table_bin[key] # to find the corresponding list in the r0_table_bin hash table 
        r0_e = r0_v[i] # to find the corresping element on r0_table_bin
        if r0_e > hll_d[value[i]]: # substitute the value if it is greater than the previous
            hll_d[value[i]]=r0_e
        i += 1

In [12]:
print(hll_d)
# the elements in the dictionary are the highest sequence of right zeroes found in each subgroup

{0: 14, 1: 14, 2: 21, 3: 15, 4: 14, 5: 15, 6: 13, 7: 13, 8: 15, 9: 15, 10: 13, 11: 14, 12: 15, 13: 17, 14: 15, 15: 16, 16: 15, 17: 14, 18: 11, 19: 13, 20: 17, 21: 14, 22: 15, 23: 13, 24: 15, 25: 15, 26: 15, 27: 13, 28: 14, 29: 15, 30: 14, 31: 14}


In [13]:
list_d = [] # we make a list from the dictionary
for key, value in hll_d.items():
    if value != 0:      
        list_d.append(value) # to drop zeros to avoid errors in the estimate

#### Now we can estimate the number of  different elements in the table and the relative error.
#### We have chosen m = 32 subgroups, to wich corresponds a coefficient alpha = 0.697

In [14]:
print('The estimated number of different elements in the table is ', int(32**2 * 0.697 * (1 / sum([2**(-x) for x in list_d]))))
error = 1.04 / np.sqrt(32)
print('The estimated error is ', ("%.2f" % error))

The estimated number of different elements in the table is  329328
The estimated error is  0.18
